# CSCI E-109a Final Project

## Team Members: Melissa Curran, Alexander Dubitskiy, Deepika Sinha, Liangliang Zhang

----------

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Introduction

Here is a chart of Microsoft adjusted closing price for the past 20 years:

In [2]:
%%html

<img src="images/msft_adj_close.png", width=369, height=279>

If we only use this chart to draw our conclusions about the company, we could say that Microsoft enjoyed a rapid stock growth in the late nineties, followed by almost 13 years of stagnation, and finally enjoying a very impressive stock rally in the past few years. That is just the big picture, but if we look at the data in more detail, we’ll see many more local ups and downs during each month, week, and even each day (sadly, for this project we do not have the intraday data).

Our goal for this project is to develop a system of detecting anomalies by observing the daily stock prices one by one (in so-called ‘online’ mode). We also would like to be able to do it with some configurable confidence level.

At this point, we would like the reader to think about _anomalies_ as unusual price levels or changes where we cannot explain why the price went up or down or stayed flat. We will expand upon this in the sections below.

----------

### What is an Anomaly?

In [3]:
%%html

<img src="images/outlaw.png", width=136, height=229>

It is very easy to define what an outlier is if we know the underlying distribution shape. Let’s say we know our system generates observations from a normal distribution with a mean of 0.0 and standard deviation of 1.0, and we observe a value of 3.0. Knowing normal distribution properties, we could calculate that the probability of observing this value (or larger) is around 0.0013. It is quite low, and if we are willing to set our threshold at 0.025 (completely arbitrarily), then we could declare our observation an outlier (because the probability of our observation is lower than the threshold).

This particular observation can also be called an “anomaly,” and in general, the terms “anomaly” and “outlier” are very often used interchangeably.

We’d like to use the term “anomaly” to describe something different from “outlier.”

Our null hypothesis is that we observe data points generated from a normal distribution with a mean of 0.0 and standard deviation of 1.0. We set the threshold at 0.025. That means that we consider any values less than – 1.96 or more than +1.96 to be outliers.

Even if we don’t know anything about the underlying distribution, we could potentially use heuristic methods to define outliers. For example, in the Tukey method, if $value < Q1 - 1.5*IQR$ or $value > Q3 + 1.5*IQR$, then it is an outlier.

In our context we’d like to use the term “anomaly” to describe a very different concept from “outlier.”  

We’d like to use the term “anomaly” as a point in time where we think our null hypothesis does not stand any more. Let’s say we observed quite a few outliers, and other values which may be not outliers but still quite extreme considering our null hypothesis. At some point in time, we should reject our null hypothesis and conclude that we detected an anomaly. 

Even more generally, we’d like to call an “anomaly” a point of time where we believe the underlying distribution has changed even if we do not exactly know the shape of the distribution, so our null hypothesis becomes “there is no change in the underlying distribution.”

#### Review of Methods of Anomaly Detection in Time Series Data

Existing anomaly detection methods for stock market data can be classified based on how they transform the data prior to anomaly detection and their process of identifying anomalies. (Summary below adapted from Golmohammadi 2015)

**Transformations:** Before anomaly detection begins, data must be transformed in order to handle high dimensionality, scaling, and noise and to achieve computational efficiency. This can be done in three ways: aggregation, discretization, and signal processing.

- **_Aggregation_** involves dimensionality reduction by aggregating consecutive values, typically by representing them by their average.

- **_Discretization_** involves converting the time series into a discrete sequence of finite alphabets, which allows you to use existing symbolic sequence anomaly detection algorithms and also to improve computational efficiency.

- **_Signal processing_** involves mapping the data to a different space in order to make it easier to detect outliers and to reduce dimensionality.

**Processes of Identifying Anomalies:** The current processes of identifying anomalies can be categorized into five groups: window based, proximity based, prediction based, hidden markov model (HMM) based, and segmentation based.

- **_Window Based:_** The time series is divided into evenly sized windows of subsequences and the distance from that sliding window to the windows in the training database determines the anomaly score. Selection of the optimal window size must be done carefully and take into account the length of the anomalous subsequence. This method can be computationally expensive (O((nl)2)), where n is the number of samples in the training and testing datasets and l is the average length of the time series.

- **_Proximity Based:_** This method uses pairwise proximity between the testing and training time series using an appropriate distance/similarity kernel. The similarity measure is then used to measure the distance of every two given sequences. A k-NN or clustering method is used to calculate the anomaly score. A major disadvantage is that this method can identify an anomalous time series, but it cannot pinpoint the exact location of the anomaly. It is also highly sensitive to the similarity measure used.

- **_Prediction Based:_** These assume that the normal time series is generated from a statistical process while the anomalies do not fit this process. However, the length of history used for prediction is very influential in locating anomalies. In addition, these methods perform very poorly when the time series was not generated from a statistical process.

- **_Hidden Markov Model (HMM) Based:_** In these models, the training dataset is used to build a hidden Markov model (HMM), which is then used to probabilistically assign an anomaly score to a given test time series. However, if the underlying time series is not generated from an HMM, it will perform poorly.

- **_Segmentation Based:_** The time series is divided into segments. These methods assume that there is an underlying Finite State Automaton (FSA) that models the normal time series, and anomalies are detected when segments do not fit the FSA. However, the segmentation procedure may obscure anomalies.



|  | Aggregation | Discretization | Signal Processing |
|-----------------------|
| Window Based | kNN (Chandola 2009), SVM (Ma 2003b), (Golmohammadi 2015) | kNN (Chandola 2009) |  |
| Proximity Based | PCAD (Protopapas 2006, Rebbapragada 2009), Martingale (Fedorova 2012, Ho 2010, Vovk 2003) |  |  |
| Prediction Based | Moving Average (Chatfield 2004), Auto Regression (Chatfield 2004), Kalman Filters (Knorn 2008), SVM (Ma 2003a) | FSA (Michael 2000) | Wavelet (Lotze 2006, Zhang 2003) |
| HMM Based | (Liu 2008) | (Qiao 2002), (Zhang 2003) |  |
| Segmentation | (Chan 2005), (Mahoney 2005), (Salvador 2005) |  |  |

----------

### Why We Detect Anomalies

Well, we tried to detect outliers in the Microsoft stock. It turned out that nothing is normally distributed (we tried adjusted closing price, closing price change from previous day, and closing price difference from 7, 30 and 120 day moving averages and also some transformations of the previous including LOG, SQRT and some others). We had to look at one of the heuristic methods, the Turkey method, and saw something like:

In [4]:
%%html

<img src="images/outliers_ma120_difference.png", width=554, height=64>

On the chart above, the red dots are outliers detected in the values representing the difference between the 120 day moving average and the closing price of the Microsoft stock. On those dates, the change in the price was somehow “extreme,” probably because they were much bigger than usual. 

We immediately realised we have a couple of problems:

1.	We need a context. Using just the closing price or its derivatives could be truly misleading. We need to include more data about general market conditions, state of the economy, and state of the company on that date to truly see if something “extreme” happened. 
2.	Detecting an outlier is of a limited use for us. It could be just a randomness in the process and we’d like to be able to capture the underlying change with some kind of confidence. 

That is the main difference between detecting outliers and anomalies: we detect outliers to exclude them from our model because they might just represent some randomness we do not want to capture. 

We detect anomalies to conclude (with some level of confidence) that something has changed in the underlying process. One way of looking at this would be to say that we have some variables not included in our model and not being observed and some of them are quite significant and their values have changed.

--------

### Anomalies in Stock Market Data

Let’s face it – there’s been no anomalies in the stock market for past 10 years, since the participants started to use very powerful computers to find any kind of discrepancies and capitalize on the findings. We understand that and we set our goals at an appropriate level.

We are likely to find anomalies because of our limited ability to collect, and process data and it is a problem but also a solution for us.

First of all, it does not undermine the method we use. Given a better data set (more variables, less aggregation), this method should work better and produce less false positives.

Second, our imperfect data gives us an opportunity to manually validate our results. After all, we are not going to have a training set, so we deal with unlabelled data and ultimately our method is unsupervised. But since our anomalies likely have some legitimate reasons (not captured in our data set) we could validate them by simply searching for any related and significant events that happened around the time of the anomaly. We should thank Google for that opportunity.

----------

### What is a Martingale?

In [5]:
%%html

<img src="images/chimp.png", width=274, height=265>

A martingale is defined as any of several systems of betting in which a player increases the stake usually by doubling each time a bet is lost (reference https://www.system-martingale.com/).

#### History of the term
1.	The concept of martingale in probability theory was introduced by Paul Pierre Levy (1935),
2.	Much of the original development of the theory was done by Doob (1949, Application of the theory of martingales; 1953, Stochastic Process).
3.	Part of the motivation for works on Martingale was to show the impossibility of successful betting strategies.

#### Why it represents confidence

?????

----------

### Our Method

In this project we apply the idea of testing exchangeability online (Vovk et al., 2003) using a martingale framework to detect concept changes in time-varying data streams.

In brief, the anomaly detection happens by performing the following steps:

1. A data point from the incoming stream is observed.
2. A strangeness measure is calculated.
3. A p-value is calculated (H0 is no concept change).
4. A randomized power martingale is constructed.
5. A test is performed using the martingale value.

#### Strangeness

We wanted our strangeness measure to: 

-	be able to handle multi-dimensional time series  
-	be able to handle unlabeled data
-	be simple and flexible 

We decided to implement two methods, both based on the well-known k-nearest neighbors algorithm:

1. Proximity based - maximum distance in the neighborhood.
2. Density based - relative density in the neighborhood.

The maximum distance in the neighborhood measure is straightforward: given a point and a neighborhood of size k, we find a distance to the furthest point away in the neighborhood.

The relative density is a bit more involved. Given a neighborhood of size k, we measure density as an inverse of the sum of all distances between the point and its neighbors. The relative density is calculated by dividing the point density by sum of densities for all the k-nearest neighbors.

#### P-Values

is calculated as:

$$p = \frac{ count(i:a_{i} > a_{n}) + \theta*count(i:a_{i} = a_{n}) }{n}$$

where $a_{i}$ is the strangeness measure for $i-th$ observation and $\theta$ is randomly chosen from $[0, 1]$

#### Martingale

is calculated as:

$$M_{n} = \epsilon p_{n}^{(\epsilon - 1)} M_{n - 1}$$

where $\epsilon$ is a float value from $[0, 1]$

----------

### How We Calculate Strangeness

#### Strangeness Measure by the Metrics

Our strangeness measures can use the following distance metrics:  

- 'euclidean' EuclideanDistance:  $sqrt(sum((x - y)^2))$
- 'manhattan' ManhattanDistance:  $sum(|x - y|)$
- 'chebyshev' ChebyshevDistance   $max(|x - y|)$

In our experiment, it is not so obvious which distance should be used and why, but when we work with high dimensional data, we should try all of them.

#### KNN-based, density, proximity

With n_neighbors = 1, the relative density measure would not be able to detect observations slowly moving away from the cluster. It looks like we should use a 'conventional' value for n_neighbors (between 3 or 5 in most cases).

The proximity measure did well in this scenario with all of the neighborhood sizes. It also looks like larger neighborhood sizes allow the measure to stay 'strange' for longer time periods (that is why it did well here).

Again, the separate cluster forming was easily detected by both measures. The relative density appears a bit more stable and tolerant to initial cluster building, but the proximity measure could potentially produce large values for longer time periods, which could be beneficial as we would like our martingale to have enough time for confidence building.

It is obvious the martingale needs to build a confidence level by receiving quite a few small p-values before the threshold is reached. That might not work very well in conjunction with KNN-based metrics as the strangeness measure might go down as soon as the new neighborhood is built.

This experiment highlights a possible weakness in our model: a small neighborhood size could prevent us from detecting underlying changes in the data stream. In this case, as soon as the number of observation in the separate cluster reached the neighborhood size, the relative density went down and the martingale did not have enough time to build the confidence.

The proximity strangeness measure seems to be the weaker option. More investigation should be done to understand where it is applicable.

#### Epsilon

The initial martingale value is $M_{0} = 1.0$ and with every new observation we calculate the martingale value as $M_{n} = \epsilon p_{n}^{(\epsilon - 1)} M_{n - 1}$, where $p$ is the p-value.

Changing the epsilon value changes how quickly the martingale rises/falls in the presence of very small or large p-values.

Let's simulate a stream of p-values perhaps indicating some change in the middle of the stream and observe changes in the martingale value for different epsilon values.

From the chart above, we could see that small epsilon values should not be used for stable streams. A martingale calculated with epsilon = 0.15 would need more then 1000 observations with p-values of 0.1 to breach a threshold of 10.

On the chart above, we see how the epsilon value plays a role in the martingale calculation. The concept change happens at step five and all the martingales reach maximum value around step 14 (at different levels). If we set the threshold at level 4.0, then a martingale with epsilon = 0.65 would detect the change at step 10 while martingale with epsilon 0.35 would detect the change at step 14. It looks like too small or too big an epsilon value might not be very sensitive to the changes, and the best performing epsilon values in this case are in 0.75 - 0.95 range.

#### Default Values

- the neighborhood consists of 3 neighbors
- density calculated relatively to total density in the neighborhood
- euclidean distance is used
- epsilon = 0.92

#### Examples of behavior

?????

- our default martingale calculation is not very sensitive to the sharp but short lived changes of the strangeness measure
- changes in the neighborhood size might have a lot of influence on the relative density calculation and the martingale value

#### Detecting Mutiple Changes

Because of the way we calculate the p-values, we will not be able to detect some sequence of the concept changes. For example, if our system changes from state A to state B, we might be able to detect the change. But the change from B back to A will be unnoticeable because the values associated with state A had been observed before. We might be able to solve this problem by introducing some kind of expiration policy on the historical values.

----------

### Method We Developed

#### How To Use

?????

#### Parameters

Our class is called KNNAnomalyDetector and it accepts the following parameters:

    threshold - float, optional (default value 2.0)
        an anomaly is reported when the threshold is breached
    epsilon - float, optional (default value 0.92)
        Value from 0.0 to 1.0, used in the martingale calculation
    n_neighbors - int, optional (default value 3)
        The neighborhood size
    metric - string, optional (default value 'euclidean')
        The distance metric used to calculate the k-neighbors for each sample point
    method - string , 'density' or 'proximity,' default value 'density'
        Method of calculating the strangeness function. 
        When 'density' the strangeness is calculated as density around the point 
        divided by total density around the neighborhood
        When 'proximity' the strangeness is calculated as maximum distance in the the neighborhood
    anomaly - string , 'level' or 'change,' default value 'level'
        Method of using the threshold
        When 'level,' the martingale value is compared with the threshold
        When 'change,' the change in martingale value is compared with the threshold

----------

### Our Dataset

All the data has been collected from open and free sources.

**Microsoft Stock**
We obtained Microsoft adjusted closing price values from Yahoo Finance for every trading day since 1995. In our dataset, it is represented by the variable 'Adj Close.'

**S&P500**
We obtained S&P500 closing price values from http://www.cboe.com/micro/buywrite/dailypricehistory.xls for every trading day since 1986 (data from Yahoo Finance was only available from 2005). In our dataset, it is represented by the variable 'SP500.'

**10-Year Treasury Constant Maturity Rate**
The data was obtained from https://fred.stlouisfed.org/series/DGS10 for every trading day since 1989. In our dataset, it is represented by the variable 'DGS10.'

**Federal Funds Rate**
The data was obtained from https://fred.stlouisfed.org/series/FEDFUNDS for every calendar month since 1954. We adjust the data frequency to trading day by padding it forward. In our dataset, it is represented by the variable 'FEDFUNDS.'

**Microsoft Earnings**
The data was obtained from https://www.microsoft.com/en-us/Investor/earnings/trended/quarterly-income-statements.aspx in for every quarter from Q3 1995. In our dataset, it is represented by the variables ‘Revenue,’ ‘Gross Margin,’ ‘Operating Income,’ and ‘Diluted EPS.’ We adjust the data frequency to trading day by padding it forward.

**Acquisition History**
The data was obtained from https://www.microsoft.com/en-us/Investor/acquisition-history.aspx and consists of the press release date and the company name. It was manually copied and converted into CSV format. In our dataset, it is represented by the variable 'Acquisition,' which takes a value of one if a press release happened on each given date and zero otherwise.

**Investment History**
The data was obtained from https://www.microsoft.com/en-us/Investor/investment-history.aspx and consists of the press release date and the company name. It was manually copied and converted into CSV format. In our dataset it is represented by the variable 'Investment', which takes a value of one if a press release happened on each given date and zero otherwise.

**SEC Filings**
The SEC filings data was obtained from https://www.microsoft.com/en-us/Investor/sec-filings.aspx. It was manually copied and converted into CSV format. It was available from 1994 and consists of the filing date and the document type. In our dataset, it is represented by a set of dummy variables. The variable name is the document type and the value is one if the document type was filed on the date, zero otherwise.

**Data Refresh**
Only the adjusted closed price is automatically refreshed and includes the newest data. All other sources have to be manually updated by downloading relevant files from the sources.

In [6]:
%%html

<img src="images/msft_adj_close.png", width=554, height=419>

<img src="images/outliers_first_diff.png", width=554, height=288>

<img src="images/outliers_second_diff.png", width=554, height=288>

<img src="images/one_class_svm.png", width=554, height=288>

----------

### Detecting Anomalies

In [7]:
%%html

<img src="images/gates.png", width=366, height=206>

We ran our algorithm on the 120 days of Microsoft stock closing price data. The graph below shows the change in closing price (blue), the value of the martingale (green), and the anomaly that was detected (red dot).

In [8]:
%%html

<img src="images/results1.png", width=831, height=314>

Based on the martingale anomaly detection model, a concept change is detected on April 22 2016. At this point, the martingale value is higher than a predefined threshold (6.0). This date is consistent with the date when a significant close price drop occurs on April 22 2016. There is no significant delay on this detection.

In [9]:
%%html

<img src="images/results2.png", width=692, height=242>

If the martingale analysis was performed multiple times, a slightly different concept change date would be reported (charts for each repeats not shown). In a 10-time-repeat experiment, the resulting concept change dates range from March 31 to April 27, with the vast majority of dates from April 21 on.

The uncertain part in martingale-base anomaly detection is $\theta$, which is randomly chosen from [0,1] when calculating p. A smaller $\theta$ will result in a lower p value and a higher martingale value. Anomaly status of data points that has a martingale value around the threshold can easily be affected by the value of $\theta$.

#### News and SEC Filings in the Days Before the Concept Change Date

In [10]:
%%html

<img src="images/results4.png", width=728, height=807>

In [11]:
%%html

<img src="images/results5.png", width=242, height=102>

After reviewing the news and SEC filings on or up to 7 days before the concept change date, we found that on April 21 2016, Microsoft announced the Third Quarter Results, which included a 6% decline in fiscal third-quarter revenue to 20.5 billion and earnings of 3.8 billion which fell 25% year over year. This event is likely to be the causal factor inducing the stock price drop on April 22 2016.

There are two parameters in our model, $\epsilon$ and $n\_neighbors$, can affect whether a data point would be classified as anomaly or not. In the above analysis, $n\_neighbors=5$ and $\epsilon=0.92$ were used as default parameter values. Next, different parameter values are tried on Microsoft data.

#### Anomaly Detection with an Incresed $\epsilon=0.95$

In [12]:
%%html

<img src="images/results6.png", width=831, height=314>

#### Anomaly Detection with a Decreased $\epsilon=0.89$

In [13]:
%%html

<img src="images/results7.png", width=831, height=314>

Even a slight change on the value of $\epsilon$ will lead to a significant change in the martingale, and then affect the decision on anomaly detection. Observation in Microsoft stock data shows that lower $\epsilon$ leads to a more sensitive model, tending to classify a certain data point as anomaly, and vice versa. Increase model sensitivity would lead to higher false positive rate while decreased model sensitivity would lead to higher false negative rate.

#### Anomaly Detection with an Increased $n\_neighbors=8$

In [14]:
%%html

<img src="images/results8.png", width=831, height=314>

#### Anomaly Detection with a Decreased $n\_neighbors=3$ 

In [15]:
%%html

<img src="images/results9.png", width=831, height=314>

Alteration on n_neighbors does not have a major influence on martingale values.

In practical applications, the values of these two parameters need to be chosen carefully. In our analysis on Microsoft stock data, the default value of $n\_neighbors=5$ and $\epsilon=0.92$ seems to be reasonable regarding both accuracy and delay time.

#### Analysing Last 360 Days of MSFT Stock

In [16]:
%%html

<img src="images/results360.png", width=831, height=314>

When running our algorithm on 360 days of Microsoft stock closing price data, two concept change dates were detected, 7/21/2015 and 2/8/2016.

----------

### Possible Improvements

1.	Our class uses the standard scikit-learn implementation of the KNN-based distance calculation. When we receive a new observation, we add it to the previous observations and recalculate all distances between them. This approach is very computationally expansive and could be improved by only calculating distances between previous observations and the new one.
2.	Better handling of the multidimensional data. For example, we have both closing stock price and earnings in our data, and when both stock price and earnings are going down or up it should not be considered as ‘strange’ as when they move in the opposite directions. One way of handling this would be to use a regression model to predict the stock price using other variables as predictors and measure strangeness based on the distance between predicted and observed values.
3.	We only offer KNN-based strangeness measures at present. A better choice of the strangeness measures should be offered, including SVM-based (using One Class SVM from scikit-learn) and decision tree-based (using Isolation Forest from scikit-learn).

-------

### Conclusion

We have implemented a martingale-based concept change detection method and applied it to a multi-dimensional dataset with Microsoft stock price and other variables describing the state of the company and the economy.

We have successfully detected anomalies (concept changes) in online mode using past 120 and 360 days of our data and manually validated them.

We are confident our implementation can be used to detect anomalies in multidimensional time series data from different sources, but we also understand the limitations as described in the ‘Possible Improvements’ section.

----------

### References

Chan, P., & Mahoney, M. (2005). Modeling multiple time series for anomaly detection. Data Mining, Fifth IEEE International Conference on, 8 pp.

Chandola, V., Cheboli, D., & Kumar, V. (2009) Detecting anomalies in a time series database. Technical Report, 1-12.

Chatfield, C. (2004). The analysis of time series : An introduction (6th ed., Texts in statistical science). Boca Raton, FL: Chapman & Hall/CRC.

Fedorova, V., Gammerman, A., Nouretdinov, I., & Vovk, V. (2012). Plug-in martingales for testing exchangeability on-line. Proceedings of the 29th International Conference on Machine Learning, Edinburgh, Scotland, UK.

Golmohammadi, K., & Zaiane, O. (2015). Time series contextual anomaly detection for detecting market manipulation in stock market. Data Science and Advanced Analytics (DSAA), 2015. 36678 2015. IEEE International Conference on, 1-10.

Ho, S-S., & Wechsler, H. (2010). A Martingale Framework for Detecting Changes in Data Streams by Testing Exchangeability. Pattern Analysis and Machine Intelligence, IEEE Transactions on, 32(12), 2113-2127.

Knorn, F., & Leith, D. (2008). Adaptive Kalman Filtering for anomaly detection in software appliances. INFOCOM Workshops 2008, IEEE, 1-6.

Liu, Z., Yu, J. X., Chen, L., & Wu, D. (2008). Detection of shape anomalies: A probabilistic approach using hidden Markov Models. 1325-1327.

Lotze, T., Shmueli, G., Murphy, S., & Burkom, H. (2006) A wavelet-based anomaly detector for early detection of disease outbreaks. Work. Mach. Learn. Algorithms Surveill. Event Detect. 23rd Intl Conf. Mach. Learn.
Ma, Junshui, & Perkins, Simon. (2003a). Online novelty detection on temporal sequences. 613-618.

Ma, J., & Perkins, S. (2003b). Time-series novelty detection using one-class support vector machines. Neural Networks, 2003. Proceedings of the International Joint Conference on, 3, 1741-1745.

Mahoney, M., & Chan, P. (2005) Trajectory boundary modeling of time series for anomaly detection. KDD Workshop on Data Mining Methods for Anomaly Detection.

Michael, C., & Ghosh, A. (2000) Two state-based approaches to program-based anomaly detection. Proceedings 16th Annual Computer Security Applications Conference (ACSAC’00), 21-30.

Protopapas, P., Giammarco, J., Faccioli, L., Struble, M., Dave, R., & Alcock, C. (2006). Finding outlier light curves in catalogues of periodic variable stars. Monthly Notices of the Royal Astronomical Society, 369(2), 677-696.

Rebbapragada, U., Protopapas, P., Brodley, C., & Alcock, C. (2009) Finding anomalous periodic time series. Machine Learning, 74(3), 281-313.

Salvador, S., & Chan, P. (2005). Learning states and rules for detecting anomalies in time series. Applied Intelligence, 23(3), 241-255.

Shafer, G., and Vovk, V. (2001) Probability and Finance: It’s only a Game! Wiley-Interscience.

Qiao, Xin, Bin, & Qiao, Y. (2002). Anomaly intrusion detection method based on HMM. Electronics Letters, 38(13), 663-664.

Vovk, V. (1993) A Logic of Probability, with Application to the Foundations of Statistics, Journal of the Royal Statistical Society, Series B (Methodological), Vol. 55, 2, 317-351.

Vovk, V., Shafer, Glenn, & Gammerman, A. (2005). Algorithmic Learning in a Random World.

Vovk, V., Nouretdinov, I., Gammerman, A. (2003) Testing exchangeability on-line. Proceedings of the Twentieth International Conference on Machine Learning (ICML-2003), Washington DC.

Zhang, X., Fan, P., & Zhu, Z. (2003). A new anomaly detection method based on hierarchical HMM. Parallel and Distributed Computing, Applications and Technologies, 2003. PDCAT'2003. Proceedings of the Fourth International Conference on, 249-252.

Zhang, J., Tsui, F., Wagner, M., & Hogan, W. (2003). Detection of outbreaks from time series data using wavelet transform. AMIA Annual Symposium Proceedings. AMIA Symposium, 748-52.
